In [4]:
#make typing paths easier within other lines of code
#to find your path of specific content, hover on file and press
#the 3 buttons then press copy path
import pandas as pd

TFIDF_ENCODING_PATH = 'data/models/tfidf_encodings.pkl'
TFIDF_MODEL_PATH = 'data/models/tfidf.pkl'

RECIPES_PATH = 'data/kaggle/RAW_recipes.csv'
PARSED_PATH = 'data/kaggle/df_parsed.csv'

In [41]:
#if we take out from HERE (THIS CELL) TO
import nltk
vocabulary = nltk.FreqDist()
recipe_df = pd.read_csv(RECIPES_PATH)


In [32]:
recipe_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [9]:
# only get the data that we need : name, id, ingredients  
new_recipe_df = recipe_df.copy()
new_columns = ['name' ,'id' , 'ingredients']
new_recipe_df = new_recipe_df.drop([c for c in new_recipe_df.columns if c not in new_columns], axis='columns')
new_recipe_df.head()

,name,id,ingredients
0,arriba baked winter squash mexican style,137739,"['winter squash', 'mexican seasoning', 'mixed ..."
1,a bit different breakfast pizza,31490,"['prepared pizza crust', 'sausage patty', 'egg..."
2,all in the kitchen chili,112140,"['ground beef', 'yellow onions', 'diced tomato..."
3,alouette potatoes,59389,"['spreadable cheese with garlic and herbs', 'n..."
4,amish tomato ketchup for canning,44061,"['tomato juice', 'apple cider vinegar', 'sugar..."


In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

# Tfidf needs unicode or string types
new_recipe_df['ingredients'] = new_recipe_df.ingredients.values.astype('U')
# TF-IDF feature extractor
tfidf = TfidfVectorizer()
tfidf.fit(new_recipe_df['ingredients'])
tfidf_recipe = tfidf.transform(new_recipe_df['ingredients'])
# save the tfidf model and encodings
with open(TFIDF_MODEL_PATH , "wb") as f:
     pickle.dump(tfidf, f)
with open(TFIDF_ENCODING_PATH, "wb") as f:
     pickle.dump(tfidf_recipe, f)

In [23]:
new_recipe_df['ingredients']

0         ['winter squash', 'mexican seasoning', 'mixed ...
1         ['prepared pizza crust', 'sausage patty', 'egg...
2         ['ground beef', 'yellow onions', 'diced tomato...
3         ['spreadable cheese with garlic and herbs', 'n...
4         ['tomato juice', 'apple cider vinegar', 'sugar...
                                ...                        
231632    ['celery', 'onion', 'green sweet pepper', 'gar...
231633    ['paprika', 'salt', 'garlic powder', 'onion po...
231634    ['hard-cooked eggs', 'mayonnaise', 'dijon must...
231635    ['butter', 'eagle brand condensed milk', 'ligh...
231636    ['granulated sugar', 'shortening', 'eggs', 'fl...
Name: ingredients, Length: 231637, dtype: object

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

# load in tdidf model and encodings
with open(TFIDF_ENCODING_PATH, 'rb') as f:
     tfidf_encodings = pickle.load(f)
with open(TFIDF_MODEL_PATH, "rb") as f:
     tfidf = pickle.load(f)

ingredients = '["bacon", "eggs", "bread"]'

# use our pretrained tfidf model to encode our input ingredients
ingredients_tfidf = tfidf.transform([ingredients])
# calculate cosine similarity between actual recipe ingreds and test ingreds 
cos_sim = map(lambda x: cosine_similarity(ingredients_tfidf, x), tfidf_encodings)
scores = list(cos_sim)

In [33]:
def get_recommendations(N, scores):
    # load in recipe dataset
    # df_recipes = pd.read_csv(PARSED_PATH)
    # order the scores with and filter to get the highest N scores
    top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]
    # create dataframe to load in recommendations
    recommendation = pd.DataFrame(columns = [])
    count = 0
    for i in top:
        recommendation.at[count, 'name'] = recipe_df['name'][i]
        recommendation.at[count, 'id'] = recipe_df['id'][i]
        recommendation.at[count, 'ingredients'] = recipe_df['ingredients'][i]
        recommendation.at[count, 'nutrition'] = recipe_df['nutrition'][i]
        recommendation.at[count, 'score'] = "{:.3f}".format(float(scores[i]))
        count += 1
    return recommendation

In [34]:
top100_recipes = get_recommendations(100, scores)
top100_recipes.head()

,name,id,ingredients,nutrition,score
0,gypsy bacon,398059.0,"['bacon', 'bread']","[66.5, 1.0, 4.0, 7.0, 3.0, 0.0, 4.0]",0.898
1,eggs lookin at you,246216.0,"['butter', 'bread', 'bacon', 'eggs', 'salt & p...","[589.4, 66.0, 11.0, 39.0, 43.0, 87.0, 8.0]",0.890
2,eggs in muffin cups,40092.0,"['eggs', 'white bread', 'bacon', 'butter']","[181.5, 21.0, 3.0, 10.0, 16.0, 28.0, 1.0]",0.863
3,egg in the nest,216223.0,"['bread', 'egg', 'bacon']","[347.6, 40.0, 5.0, 25.0, 26.0, 42.0, 4.0]",0.792
4,grandpa farrell s scrambled eggs,123977.0,"['bacon', 'eggs']","[252.9, 31.0, 1.0, 14.0, 30.0, 33.0, 0.0]",0.779


In [64]:
# parser to chnage the string to an array
arry_string = top100_recipes['nutrition'][0]
arry_string = arry_string[1:len(arry_string)-1]
arry_string = arry_string.split(",")
arry_string = [float(x) for x in arry_string]
arry_string

def parse_nutrition(row_no):
    arry_string = top100_recipes['nutrition'][row_no]
    arry_string = arry_string[1:len(arry_string)-1]
    arry_string = arry_string.split(",")
    arry_string = [float(x) for x in arry_string]
    return arry_string 


In [66]:
# filter: alergies, cruisines, nutritional values

# nutritional values 
# column : Nutrition information (calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV) , and carbohydrates (PDV))

# cleaning : split up the nutritions  and turn into columns
new_columns = ['calories', 'total fat (PDV)', 'sugar (PDV)' , 'sodium (PDV)' , 'protein (PDV)' , 'saturated fat (PDV)' , 'carbohydrates (PDV)']

for i in range(len(top100_recipes)):
    nutrition_series = parse_nutrition(i)
    print(nutrition_series)





[66.5, 1.0, 4.0, 7.0, 3.0, 0.0, 4.0]
[589.4, 66.0, 11.0, 39.0, 43.0, 87.0, 8.0]
[181.5, 21.0, 3.0, 10.0, 16.0, 28.0, 1.0]
[347.6, 40.0, 5.0, 25.0, 26.0, 42.0, 4.0]
[252.9, 31.0, 1.0, 14.0, 30.0, 33.0, 0.0]
[501.4, 54.0, 10.0, 30.0, 36.0, 47.0, 8.0]
[138.0, 8.0, 5.0, 8.0, 16.0, 8.0, 4.0]
[629.5, 65.0, 4.0, 75.0, 70.0, 131.0, 8.0]
[749.1, 81.0, 46.0, 56.0, 58.0, 85.0, 12.0]
[457.1, 54.0, 9.0, 28.0, 41.0, 62.0, 4.0]
[156.8, 11.0, 5.0, 10.0, 16.0, 14.0, 4.0]
[216.8, 25.0, 1.0, 19.0, 28.0, 27.0, 0.0]
[796.5, 92.0, 11.0, 53.0, 62.0, 109.0, 10.0]
[460.4, 56.0, 4.0, 42.0, 36.0, 70.0, 4.0]
[184.1, 13.0, 5.0, 13.0, 21.0, 19.0, 4.0]
[334.3, 27.0, 20.0, 25.0, 24.0, 31.0, 10.0]
[548.2, 65.0, 8.0, 45.0, 28.0, 69.0, 8.0]
[706.5, 84.0, 7.0, 46.0, 65.0, 124.0, 6.0]
[407.8, 45.0, 8.0, 32.0, 18.0, 59.0, 8.0]
[108.1, 12.0, 0.0, 5.0, 14.0, 13.0, 0.0]
[216.3, 25.0, 1.0, 11.0, 28.0, 27.0, 0.0]
[3116.2, 471.0, 0.0, 236.0, 157.0, 510.0, 1.0]
[1038.7, 157.0, 0.0, 78.0, 52.0, 170.0, 0.0]
[193.3, 17.0, 5.0, 11.0,